In [12]:
import pandas as pd
import sqlalchemy as sa
import numpy as np
import json

In [13]:
%run ../toxref_mysql_connect.py
# print(connect.__doc__)
con = connect()  

In [14]:
effect_profile_id=2
pods = pd.read_sql("pod",con).merge(pd.read_sql("pod_tg_effect",con))

/share/home/kpaulfri/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1287, "'@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead")
  result = self._query(query)


In [15]:
studies = pd.read_sql("study",con)

In [16]:
chemicals = pd.read_sql("chemical",con)

In [17]:
effect_profile_group = pd.read_sql("effect_profile_group",con)

In [18]:
effects = pd.read_sql("endpoint",con).merge(pd.read_sql("effect",con)).merge(pd.read_sql("tg_effect",con))

In [19]:
pods = chemicals.merge(pods).merge(effect_profile_group[effect_profile_group.effect_profile_id==effect_profile_id]).merge(effects,on="tg_effect_id")

In [20]:
pods["effect"]=pods[["endpoint_category","endpoint_type","endpoint_target","effect_desc"]].apply(";".join,axis=1)

In [21]:
chemical_level_pods = pd.ExcelWriter(f"chemical_level_pods_profile{effect_profile_id}.xlsx")
pods[pods.study_id.isnull()][["dsstox_substance_id","casrn","preferred_name","pod_type",
      "qualifier","pod_value","pod_unit","mg_kg_day_value","dose_level",
      "max_dose_level","group_name","endpoint_category",
      "endpoint_type","endpoint_target","effect_desc","effect"]].drop_duplicates().groupby(
    ["dsstox_substance_id","casrn","preferred_name","pod_type",
      "qualifier","pod_value","pod_unit","mg_kg_day_value","dose_level",
      "max_dose_level","group_name"]).effect.apply("|".join).sort_index().to_excel(chemical_level_pods,merge_cells=False)
chemical_level_pods.save()

In [22]:
study_level_pods = pd.ExcelWriter(f"study_level_pods_profile{effect_profile_id}.xlsx")
pods[pods.study_id.isnull()==False][["dsstox_substance_id","casrn","preferred_name","study_id","pod_type",
      "qualifier","pod_value","pod_unit","mg_kg_day_value","dose_level",
      "max_dose_level","group_name","endpoint_category",
      "endpoint_type","endpoint_target","effect_desc","effect"]].drop_duplicates().groupby(
    ["dsstox_substance_id","casrn","preferred_name","study_id","pod_type",
      "qualifier","pod_value","pod_unit","mg_kg_day_value","dose_level",
      "max_dose_level","group_name"]).effect.apply("|".join).sort_index().to_excel(study_level_pods,merge_cells=False)
study_level_pods.save()